# Getting Started

This notebook walks you through executing a `hello_world` application written with RADICAL-Pilot (RP) and locally executed on a GNU/Linux operating system. The application consists of a Bag of Tasks with heterogeneous requirements: different number of CPU cores/GPUs and different execution time for each task. In this simple application, tasks have no data requirements but see the [Data Staging](tutorials/staging_data.ipynb) tutorial for how to manage data in RP.

<div class="alert alert-warning">
    
__Warning:__ We assume you understand what a `pilot` is and how it enables to concurrently and sequentially execute compute tasks on its resources. See our [Brief Introduction to RP](https://radical-cybertools.github.io/presentations/rp_internals.mp4) video to familiarize yourself with the architectural concepts and execution model of a pilot. 

</div>

## Installation

<div class="alert alert-warning">
    
__Warning:__ RP **must** be installed in a Python environment. RP will **not** work properly when installed as a system-wide package. You **must** create and activate a virtual environment before installing RP. 

</div>

You can create a Python environment suitable to RP using [Virtualenv](https://virtualenv.pypa.io/en/latest/), [Venv](https://docs.python.org/3/library/venv.html) or [Conda](https://docs.conda.io/projects/conda/en/latest/index.html). Once created and activated a virtual environment, RP is a Python module installed via [pip](https://pip.pypa.io/en/latest/), Conda, or [Spack](https://spack.readthedocs.io/en/latest/).

<div class="alert alert-info">
    
__Note:__ Please see [using environment variables](envs.rst) with RP for more options and detailed information. That will be especially useful when executing RP on [supported high performance computing (HPC) platforms](supported.rst).

</div>

### Virtualenv

```shell
virtualenv ~/ve_rp
. ~/ve_rp/bin/activate
pip install radical.pilot
```

### Venv

```shell
python -m venv ~/ve_rp
. ~/ve_rp/bin/activate
pip install radical.pilot
```

### Conda

If there is no `conda` pre-installed, here is a distilled set of commands to install Miniconda on a GNU/Linux x86_64 OS. Find more (and possibly updated) information on the official [Conda documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/index.html))

```shell
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ./miniconda.sh
chmod +x ./miniconda.sh
./miniconda.sh -b -p ./conda
source ./conda/bin/activate
```

Once Conda is available:

```shell
conda create -y -n ve_rp python=3.9
conda activate ve_rp
conda install -y -c conda-forge radical.pilot
```

### Spack

If there is no `spack` pre-installed, here is a distilled set of commands to install Spack on a GNU/Linux x86_64 OS. Find more (and possibly updated) information on the official [Spack documentation](https://spack.readthedocs.io/en/latest/).

```bash
git clone https://github.com/spack/spack.git
. spack/share/spack/setup-env.sh
```

Once Spack is available:

```shell
spack env create ve_rp
spack env activate ve_rp
spack install py-radical-pilot
```

<div class="alert alert-info">
    
__Note:__  It is recommended to use [PYTHONNOUSERSITE](https://docs.python.org/3/using/cmdline.html#envvar-PYTHONNOUSERSITE) environment variable before activating your virtual environment to prevent user site packages from interfering: `export PYTHONNOUSERSITE=True`.

</div>

### Check the installed version

Often, we need to know what version of RP we installed. For example, you will need to know that when opening a [support ticket](https://github.com/radical-cybertools/radical.pilot/issues) with the RADICAL development team. 

We install a command with RP that prints information for all the installed [RADICAL Cybertools](https://radical-cybertools.github.io/):

In [1]:
!radical-stack


  python               : /home/merzky/radical/radical.pilot.devel/ve3/bin/python3
  pythonpath           : /home/merzky/projects/flux/install/lib/python3.12/site-packages:
  version              : 3.13.3
  virtualenv           : /home/merzky/radical/radical.pilot.devel/ve3

  radical.analytics    : 1.103.0-v1.102.0-52-g795f7da@devel
  radical.gtod         : 1.103.0-v1.102.0-1-g3dc2d07@devel
  radical.pilot        : 1.103.0-v1.102.0-184-gd49d1918b@fix/flux
  radical.utils        : 1.103.0-v1.102.0-60-g8f1b6ab6@fix/flux

  rc.process           : 0.11.0



## Write your first application

RP executes in batch mode:

* Write an application using RP API.
* Launch that application.
* Wait a variable amount of time doing something else.
* When the application exits, come back, collect and check the results.

Each RP application has a distinctive pattern:

1. Create a session
1. Create a pilot manager
1. Describe the pilot on which you want to run your application tasks: 
    - Define the platform on which you want to execute the application
    - Define the amount/type of resources you want to use to run your application tasks.
1. Assign the pilot description to the pilot manager
1. Create a task manager
1. Describe the computational tasks that you want to execute:
    - Executable launched by the task
    - Arguments to pass to the executable command if any
    - Amount of each type of resource used by the executable, e.g., CPU cores and GPUs
    - When the executable is MPI/OpenMP, number of ranks for the whole executable, number of ranks per core or GPU
    - Many other parameters. See the [API specification](apidoc.rst) for full details
1. Assign the task descriptions to the task manager
1. Submit tasks for execution
1. Wait for tasks to complete execution

Some of RP behavior can be configured via environment variables. RP's progression bar does not work properly with Jupyter notebooks. Thus, you may want to set it to FALSE.

In [2]:
%env RADICAL_REPORT_ANIME=FALSE

env: RADICAL_REPORT_ANIME=FALSE


As with every Python application, first you import all the required modules.

In [3]:
import radical.pilot as rp

### Enable user feedback

As RP implements a batch programming model, by default, it returns a minimal amount of information. After submitting the tasks for execution, RP will remain silent until all the tasks have completed. In practice, when developing and debugging your application, you will want more feedback. We wrote a reporter module that you can use with RP and all the other RADICAL-Cybertools.

To use the reporter:

* Configure RP by exporting a shell environment variable.

    ```shell
    export RADICAL_PILOT_REPORT=TRUE
    ```

* Import `radical.utils`, create a reporter and start to use it to print meaningful messages about the state of the application execution.

<div class="alert alert-info">
    
__Note:__ See our tutorial about [Profiling a RADICAL-Pilot Application](tutorials/profiling.ipynb) for a guide on how to trace and profile your application.

</div>

<div class="alert alert-info">

__Note:__ See our tutorial about [Debugging a RADICAL-Pilot Application](tutorials/debugging.ipynb) for a guide on how to debug your application.
    
</div>

In [4]:
import radical.utils as ru

report = ru.Reporter(name='radical.pilot')
report.title('Getting Started (RP version %s)' % rp.version)

 Getting Started (RP version 1.102.0)                                           


### Creating a session

[rp.Session](apidoc.html#radical.pilot.Session) is the root object of all the other objects of RP.

In [5]:
session = rp.Session()

new session: 

[rp.session.thinkie.merzky.020360.0003]

                           \
zmq proxy  : 

[tcp://192.168.0.247:10001]

                                      ok


### Creating a pilot manager

You need to manage the resources you will acquire with a pilot either locally or, more commonly and usefully, on a supported HPC platform. An instance of [rp.PilotManager](apidoc.html#radical.pilot.PilotManager) attached to your `session` will do that for you.

<div class="alert alert-info">
    
__Note:__ One [rp.PilotManager](apidoc.html#radical.pilot.PilotManager) can manage multiple pilots. See our tutorial about [Using Multiple Pilots with RADICAL-Pilot](tutorials/multiple_pilots.ipynb) to see why and how.

</div>

In [6]:
pmgr = rp.PilotManager(session=session)

create pilot manager

                                                          ok


### Configuring pilot resources

You can use a dictionary to specify location, amount and other properties of the resources you want to acquire with a pilot; and use that dictionary to initialize a [rp.PilotDescription](apidoc.html#radical.pilot.PilotDescription) object. See the [Configuration](tutorials/configuration.ipynb) tutorial for more details.

In this example, we want to run our `hello_world` application on our local GNU/Linux, for not more than 30 minutes and use 2 cores. 

<div class="alert alert-warning">
    
__Warning:__ We choose a 30 minutes runtime, but the application could take less or more time to complete. 30 minutes are the upper bound but RP will exit as soon as all tasks have reached their final state (`DONE`, `CANCELED`, `FAILED`). Conversely, RP will always exit once the runtime expires, even if some tasks still need to be executed.

</div>

<div class="alert alert-info">
    
__Note:__ We could choose to use as many CPU cores as we have available on our local machine. RP will allocate all of them, but it will use only the cores required by the application tasks. If all the tasks together require fewer cores than those available, the remaining cores will go unused. Conversely, if there are more tasks that cores, RP will schedule each task as soon as the required amount of cores becomes available. In this way, RP will maintain the available resources as busy as possible and the application tasks will run both concurrently and sequentially, depending on resource availability.

</div>

<div class="alert alert-info">
    
__Note:__ `'exit_on_error': False` allows us to compile this notebook without errors. You should probably not use it with a standalone RP application.

</div>

In [7]:
pdesc = rp.PilotDescription({'resource'     : 'local.localhost',
                             'project'      : None,
                             'queue'        : None,
                             'cores'        : 4,
                             'gpus'         : 0,
                             'runtime'      : 30,  # pilot runtime minutes
                             'exit_on_error': False})

### Submitting the pilot

We now have a pilot manager, we know how many resources we want and on what platform. We are ready to submit our request!

<div class="alert alert-info">
    
__Note:__ On a supported HPC platform, our request will queue a job into the platform's batch system. The actual resources will become available only when the batch system schedules the job. This is not under the control of RP and, barring reservation, the actual queue time will be unknown.

</div>

We use the [rp.PilotManager.submit_pilots()](apidoc.html#radical.pilot.PilotManager.submit_pilots) method of our pilot manager and pass to it the pilot description.

In [8]:
report.header('submit pilot')
pilot = pmgr.submit_pilots(pdesc)

# preserve pilot sandbox
pilot_sandbox  = ru.Url(pilot.pilot_sandbox).path

# wait for pilot to become ACTIVE
pilot.wait(rp.PMGR_ACTIVE)

# report pilot state
report.info('<<Pilot state:')
report.ok(f'>>{pilot.state}\n')

--------------------------------------------------------------------------------


submit pilot                                                                    


submit 1 pilot(s)


        pilot.0000   local.localhost           4 cores       0 gpus

           ok


Pilot state:

                                                         PMGR_ACTIVE


### Creating a task manager

We have acquired the resources we asked for (or we are waiting in a queue to get them) so now we need to do something with those resources, i.e., executing our application tasks. First, we create a [rp.TaskManager](apidoc.html#radical.pilot.TaskManager) and associate it to our session. That manager will take care of taking our task descriptions and sending them to our pilot so that it can execute those tasks on the allocated resources.

In [9]:
tmgr = rp.TaskManager(session=session)

create task manager

                                                           ok


### Registering the pilot with the task manager

We tell the task manager what pilot it should use to execute its tasks.

In [10]:
tmgr.add_pilots(pilot)

### Describing the application tasks

In this example, we want to run simple tasks but that require different number of CPU cores and that run for a variable amount of time. Thus, we use the executable [radical-pilot-hello.sh](https://github.com/radical-cybertools/radical.pilot/blob/master/bin/radical-pilot-hello.sh) we crafted to occupy a configurable amount of resources for a configurable amount of time. 

Each task is an instance of [rp.TaskDescription](apidoc.html#radical.pilot.TaskDescription) with some defined properties:

* `executable`: the name of the executable we want to launch with the task.
* `arguments`: the arguments to pass to `executable`. In this case, the number of seconds it needs to run for.
* `ranks`: this is the number of processes (i.e., MPI ranks) on which the task should run. See [Describing tasks in RADICAL-Pilot](tutorials/describing_tasks.ipynb#MPI-Tasks-and-Task-Resources) and the details about executing tasks that use the message passing interface (MPI).
* `cores_per_rank`: the amount of cores that each rank of the task utilizes. In our case, each task will randomly use either 1 or 2 cores have requested.

<div class="alert alert-warning">

__Warning:__ Executing MPI tasks (i.e., one with `rp.TaskDescription.ranks > 1`) requires for an MPI implementation to be available on the machine on which you will run the task. That is usually taken care of by the system administrator, but if you are managing your own cluster, you will have to install and make available one of the many MPI distributions available for GNU/Linux. 

</div>

We run 10 tasks that should be enough to see both concurrent and sequential executions on the amount of resources we requested, but not enough to clog the example.

<div class="alert alert-info">
    
__Note:__ We use the reporter to produce a progress bar while we loop over the task descriptions.

</div>

In [11]:
import os
import random

n = 10

report.progress_tgt(n, label='create')
tds = list()
for i in range(n):

    td = rp.TaskDescription()
    td.executable     = 'radical-pilot-hello.sh'
    td.arguments      = [random.randint(1, 10)]
    td.ranks          =  1
    td.cores_per_rank =  random.randint(1, 2)
    td.named_env      = 'rp'

    tds.append(td)
    report.progress()

report.progress_done()

create: 

#

#

#

#

...

#

#

#

#

#

### Submitting tasks for execution

Now that we have all the elements of the application we can execute its tasks. We submit the list of application tasks to the task manager that, in turn, will submit them to the indicated pilot for execution. Upon receiving the list of task descriptions, the pilot will schedule those tasks on its available resources and then execute them.

<div class="alert alert-info">
    
__Note:__ For RP, tasks are black boxes, i.e., it knows nothing about the code executed by the task. RP just knows that a task has been launched on the requested amount of resources, and it will wait until the tasks exits. In that way, RP is **agnostic** towards task details like language used for its implementation, the type of scientific computation it performs, how it uses data, etc. This is why RP can serve a wide range of scientists, independent on their scientific domain.

</div>

In [12]:
report.header('submit %d tasks' % n)
tasks = tmgr.submit_tasks(tds)

--------------------------------------------------------------------------------


submit 10 tasks                                                                 


submit: 

#

#

...

#

#

#

#

#

### Waiting for the tasks to complete

Wait for all tasks to reach a final state (`DONE`, `CANCELED` or `FAILED`). This is a blocking call, i.e., the application will wait without exiting and, thus, the shell from which you launched the application should not exit either. Thus, no closing your laptop or no exiting from a remote connection without first leaving the shell running in background or using a terminal multiplexer like [tmux](https://github.com/tmux/tmux).

<div class="alert alert-info">

__Note:__ After the wait call returns, you can describe and/or submit more tasks/pilots as your RP session will still be open.

</div>

<div class="alert alert-info">

__Note:__ You can wait for the execution of a **subset** of the tasks you defined. See [Describing tasks in RADICAL-Pilot](tutorials/describing_tasks.ipynb) for more information.

</div>

In [13]:
tmgr.wait_tasks(uids=[t.uid for t in tasks])
report.header(f'finished {len(tasks)} tasks')

task_states   = [t.state for t in tasks]
state_counter = {state: task_states.count(state) for state in set(task_states)}
for state, counter in sorted(state_counter.items()):
    report.info('<<Tasks state:')
    report.ok(f'>>({counter}) {state}\n')

wait  : 

#

#

#

#

...

                                                                              ok


--------------------------------------------------------------------------------


finished 10 tasks                                                               


Tasks state:

                                                           (10) DONE


Once the wait is finished, let us know and exit!

In [14]:
report.header('finalize')
session.close()

--------------------------------------------------------------------------------


finalize                                                                        


closing session rp.session.thinkie.merzky.020360.0003

                          \
close task manager

                                                            ok


...

                                                            \
wait for 1 pilot(s)
        

                                                                      ok


                                                                              ok


session lifetime: 58.7s

                                                       ok


## Generated Output

RP is a distributed system, even when all its components run on a single machine as with this example. RP has two main components (Client and Agent) and both store their output into a sandbox, stored at a specific filesystem location:

- **Client sandbox**: A directory created within the working directory from where the RP application was launched. The sandbox is named after the session ID, e.g., `rp.session.nodename.username.018952.0000`.
- **Agent sandbox**: A directory created at a different location, depending on the machine on which the application executes. The Agent sandbox, named `radical.pilot.sandbox`, contains the following nested directories: `<session_ID>/<pilot_ID>/<task_ID>` which represent session-sandbox, pilot-sandbox(es) and task-sandbox(es) respectively.

When running RP on localhost, the Agent sandbox is located at `$HOME/radical.pilot.sandbox`. When using a [supported HPC platform](supported.rst), the location of the Agent sandbox depends on the filesystem capabilities of the platform. You can see the pre-configured location for the Agent sandbox in the RP git repository, at [src/radical/pilot/configs/resource_*.json](https://github.com/radical-cybertools/radical.pilot/tree/master/src/radical/pilot/configs).

<div class="alert alert-warning">

__Warning:__ When executing RP on a supported HPC platform, the output file(s) of each task are saved in the task-sandbox, named after `task_ID` of that task. Without specific staging instructions (see our tutorial [Staging Data with RADICAL-Pilot](tutorials/staging_data.ipynb)), you will have to manually retrieve those files. When executing on localhost, you can retrieve them from `$HOME/radical.pilot.sandbox/<session_ID>/<pilot_ID>/<task_ID>`.

</div>

<div class="alert alert-info">

__Note:__ When enabling debugging (see our tutorial [Debugging a RADICAL-Pilot Application](tutorials/debugging.ipynb)) and/or tracing (see our tutorial [Tracing and Profiling a RADICAL-Pilot Application](tutorials/profiling.ipynb)), RP writes the debug and/or trace files in the Client and Agent sandboxes. On large/production runs, RP can produce hundreds of debug files. Please [contact](https://github.com/radical-cybertools/radical.pilot/issues) the RADICAL development team if you need further assistance.

</div>

Here are the output files of the `task.000000` of the application we just executed in this notebook:

In [15]:
!echo $pilot_sandbox
!ls $pilot_sandbox/task.000000/

/home/merzky/radical.pilot.sandbox/rp.session.thinkie.merzky.020360.0003/pilot.0000/


task.000000.err      task.000000.launch.out  task.000000.out
task.000000.exec.sh  task.000000.launch.sh   task.000000.prof
task.000000.files    task.000000.ofiles


Here is the "result" produced by `task.000000`:

In [16]:
!cat $pilot_sandbox/task.000000/task.000000.out

0 : PID     : 317769
0 : NODE    : thinkie
0 : CPUS    : 11111111111111111111
0 : GPUS    : 00
0 : RANK    : 0
0 : THREADS : 1
0 : SLEEP   : 7
stress-ng: info:  [317867] setting to a 7 secs run per stressor
stress-ng: info:  [317867] dispatching hogs: 1 cpu
stress-ng: info:  [317867] skipped: 0
stress-ng: info:  [317867] passed: 1: cpu (1)
stress-ng: info:  [317867] failed: 0
stress-ng: info:  [317867] metrics untrustworthy: 0
stress-ng: info:  [317867] successful run completed in 7.00 secs
